In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

# Set display options (optional)
pd.set_option('display.max_columns', None)

# Define the path to the dataset in the "x" folder on Google Drive
# Change the filename here if you decide to use a different one
data_path = '/content/drive/MyDrive/MLP/NPPE-1/Preprocessing/NPPE1_Preprocessing1.csv'

# Load the dataset
df = pd.read_csv(data_path)

# Display the first few rows to confirm it's loaded correctly
df.head()

,CRIM,ZN,INDUS,POLINDEX,RM,AGE,DIS,HIGHWAYCOUNT,TAX,PTRATIO,IMM,BPL,PRICE,RIVERSIDE
0,1.026769,1.429034,7.851300,1.134216,6.0,42.0,5.251911,5,279.201277,20.689586,398.811960,10.461456,22.991633,NO
1,0.848089,0.255543,6.263434,1.245993,7.0,63.0,4.305546,8,307.444529,17.465398,377.153649,11.619690,24.551055,NO
2,10.925905,0.441022,18.322960,2.824833,8.0,-2.0,2.409495,25,666.492973,20.351601,387.061355,19.366070,15.875346,NO
3,0.559027,1.041175,11.114920,0.794952,6.0,9.0,6.898669,4,305.514181,19.787314,391.778647,6.206820,23.007756,NO
4,0.905063,81.167963,3.673369,1.029030,8.0,20.0,10.246463,1,315.913960,17.360439,395.833166,10.827105,21.503177,NO


In [ ]:
# A helper function to truncate values to 3 decimal places (without rounding off)
import math
def truncate(num, digits=3):
    stepper = 10.0 ** digits
    return math.floor(num * stepper) / stepper

## Q2 How many samples are there in the dataset?

In [ ]:
df.shape[0]

4000

## Q3 What is the average house price (in lacs)?

In [ ]:
avg_price = df['PRICE'].mean()
avg_price_trunc = truncate(avg_price, 3)
print("Average house price (in lacs) [truncated to 3 decimals]:", avg_price_trunc)

Average house price (in lacs) [truncated to 3 decimals]: 24.355


## Q4 How many houses have 5 or more rooms?

In [ ]:
houses_with_5_or_more_rooms = df[df['RM'] >= 5].shape[0]
print("Number of houses with 5 or more rooms:", houses_with_5_or_more_rooms)

Number of houses with 5 or more rooms: 3953


## Q5 What is the average price of the top 10 most expensive houses (in lacs)?

In [ ]:
top10_avg_price = df.sort_values(by='PRICE', ascending=False).head(10)['PRICE'].mean()
top10_avg_price_trunc = truncate(top10_avg_price, 3)
print("Average price of the top 10 most expensive houses (in lacs) [truncated]:", top10_avg_price_trunc)

Average price of the top 10 most expensive houses (in lacs) [truncated]: 52.365


## Q6 What is the total number of missing or unknown values in the number of rooms (RM) feature?
### Hint: First inspect the unique values to decide which values are implausible.

In [ ]:
print("Unique values in RM:", df['RM'].unique())
# For instance, if implausible values are those that are less than 1:
unknown_rm = df[df['RM'] < 1].shape[0]
print("Total missing/unknown values in RM:", unknown_rm)

Unique values in RM: [ 6.  7.  8.  9. 10. -1.  5.  4. 11.]
Total missing/unknown values in RM: 40


## Q7 What is the total number of missing or unknown values in the age (AGE) feature?

In [ ]:
print("Unique values in AGE:", df['AGE'].unique())
# Assuming implausible values are those less than 0:
unknown_age = df[df['AGE'] < 0].shape[0]
print("Total missing/unknown values in AGE:", unknown_age)

Unique values in AGE: [ 42.  63.  -2.   9.  20.  95.  35.  97.  36.  78.  99.  96.  19.  73.
  86.  37.  76.  50.  32.  92. 101.   8.  94.  84.  47.  43.  65.  46.
  59.  74.  93.  80.  98. 102.  56. 100.  89.  53.  18.  71.  62.  83.
  66.   7.  77.  91.  90.  58.  57.  60.  87.  75.  34.  85.  48.  33.
  38.  30.  23.  55.  11.  14.  45.  41.  88.  16.  12.  79.  67.  39.
  49.  61.  70.  54.  10.  22.  72.  15.  51.  81.  31.  52.  44.  28.
  17.  40.  29.  69.  64.  24.  82.  26.  68.  27.  13. 103.  21.  25.
   6.   5.   3.   4.]
Total missing/unknown values in AGE: 50


## Q8 What is the total number of missing or unknown values in the RIVERSIDE feature?

In [ ]:
print("Unique values in RIVERSIDE:", df['RIVERSIDE'].unique())
# Since RIVERSIDE is supposed to be binary (0 or 1), any value not in [0, 1] can be considered implausible:
unknown_riverside = (df['RIVERSIDE'] == 'UNKNOWN').sum()
print("Total missing/unknown values in RIVERSIDE:", unknown_riverside)

Unique values in RIVERSIDE: ['NO' 'UNKNOWN' 'YES']
Total missing/unknown values in RIVERSIDE: 88


## Q9 How many houses are on riverside and were built within the last 50 years (i.e. a house 50 years old or younger)? For this question, ignore the rows that have missing values in either riverside feature or age feature.

In [ ]:
# First, filter out rows with missing/unknown values in RIVERSIDE or AGE:
mask_valid = df['RIVERSIDE'].isin(['YES', 'NO']) & df['AGE'].notna()
# Then, filter for houses on riverside and built in the last 50 years:
mask_condition = (df['RIVERSIDE'] == 'YES') & (df['AGE'] <= 50)
count_riverside_recent = df[mask_valid & mask_condition].shape[0]
print("Number of houses on riverside built within last 50 years:", count_riverside_recent)

Number of houses on riverside built within last 50 years: 53


## Q10 How many houses are near to exactly 6, 7 or 8 highways (all three inclusive)?

In [ ]:
# (Check HIGHWAYCOUNT column for values exactly 6, 7 or 8)
mask_highways = df['HIGHWAYCOUNT'].isin([6, 7, 8])
count_highways = df[mask_highways].shape[0]
print("Number of houses near exactly 6, 7 or 8 highways:", count_highways)


Number of houses near exactly 6, 7 or 8 highways: 1211


## Q11 Create a column 'CATEGORY' and divide the houses in categories as following:

- Category 1: house price <10 lacs
- Category 2: 10 lacs <= house price <20 lacs
- Category 3: 20 lacs <= house price <30 lacs
- Category 4: 30 lacs <= house price <40 lacs
- Category 5: house price >=40 lacs

Which category has the highest number of records?

In [ ]:
def assign_category(price):
    if price < 10:
        return 1
    elif price < 20:
        return 2
    elif price < 30:
        return 3
    elif price < 40:
        return 4
    else:
        return 5

df['CATEGORY'] = df['PRICE'].apply(assign_category)
category_counts = df['CATEGORY'].value_counts()
print("Category counts:\n", category_counts)
print("Category with highest number of records:", category_counts.idxmax())


Category counts:
 CATEGORY
3    2028
2    1158
4     503
5     268
1      43
Name: count, dtype: int64
Category with highest number of records: 3


## Q12 Apply Pre processing
Divide the data into training and test sets

1. Replace the respective missing or unknown values in features room count, riverside and age with np.nan.
2. Keep 30% of the data as test set.
3. Use random_state as 0
4. PRICE is the target, rest of the columns are the features.
5. Apply train test split.
Hint: look for the documentation of the usual function that divides the data into training and test datasets.

What is the number of samples in the training set?

In [ ]:
#    (Assuming any occurrence of "UNKNOWN" should be replaced.)
for col in ['RM', 'RIVERSIDE', 'AGE']:
    df[col] = df[col].replace("UNKNOWN", np.nan)

# 2. Split data into features (X) and target (y), where PRICE is target.
X = df.drop(columns=['PRICE'])
y = df['PRICE']

# 3. Apply train_test_split with 30% as test and random_state=0.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Number of samples in the training set:", X_train.shape[0])

Number of samples in the training set: 2800


## Q13 Apply following preprocessing steps:

1. Drop CATEGORY column
2. CRIM: min max scaling
3. ZN: min max scaling
4. INDUS: standard scaling
5. POLINDEX: min max scaling
6. DIS: min max scaling
7. HIGHWAYCOUNT: min max scaling
8. TAX: min max scaling
9. PTRATIO: min max scaling
10. IMM: min max scaling
11. BPL: min max scaling
12. RM: impute with median then min max scaling
13. AGE: impute with mean then min max scaling
14. RIVERSIDE: Impute with most frequent value then one hot encode.

NOTE:
1. Make sure to preprocess the features in exactly above order. Answer of Q.16 depends upon correct order of featuring processing.
2. You may have to use multiple instances of a trasnformer for this question.


How many features are there after performing above transformation?

In [ ]:
# First, drop the 'CATEGORY' column if it exists.
X_train_proc = X_train.drop(columns=['CATEGORY'], errors='ignore')
X_test_proc  = X_test.drop(columns=['CATEGORY'], errors='ignore')

# Define the groups of columns and corresponding transformers.
minmax_cols = ['CRIM', 'ZN', 'POLINDEX', 'DIS', 'HIGHWAYCOUNT', 'TAX', 'PTRATIO', 'IMM', 'BPL']

# For RM: impute with median then min-max scaling.
rm_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# For AGE: impute with mean then min-max scaling.
age_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# For INDUS: standard scaling.
# For RIVERSIDE: impute with most frequent then one-hot encode.
riverside_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# Build a ColumnTransformer to apply the transformations.
preprocessor = ColumnTransformer(transformers=[
    ('minmax', MinMaxScaler(), minmax_cols),
    ('indus', StandardScaler(), ['INDUS']),
    ('rm', rm_pipe, ['RM']),
    ('age', age_pipe, ['AGE']),
    ('riverside', riverside_pipe, ['RIVERSIDE'])
], remainder='drop')

# Fit the preprocessor on the training data and transform both training and test sets.
X_train_transformed = preprocessor.fit_transform(X_train_proc)
print("Number of features after transformation:", X_train_transformed.shape[1])

Number of features after transformation: 14


## Q14 What is the mean of the transformed test data (features only)?
Note : Compute the mean of the whole feature matrix i.e. mean of all values in the transformed test feature matrix


In [ ]:
X_test_transformed = preprocessor.transform(X_test_proc)
overall_mean = X_test_transformed.mean()  # Mean of all values in the feature matrix
print("Mean of the transformed test feature matrix:", overall_mean)

Mean of the transformed test feature matrix: 0.40315397580513196
